[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yuyan124/sd_tools/blob/main/kohya_sd_scripts.ipynb)

In [ ]:
# GPU INFO
!nvidia-smi
!nvcc -V
!free -h

In [ ]:
#@title 更新python到3.10
!apt-get install python3.10
!python -m pip uninstall pips
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.10 2
!sudo apt-get install python3.10-distutils
!wget https://bootstrap.pypa.io/pip/get-pip.py
!python get-pip.py
!python --version

In [ ]:
#@title Clone sd-scripts, 安装依赖，下载模型
#@markdown ## 模型的路径是/content/models/model.ckpt
!git clone https://github.com/kohya-ss/sd-scripts.git
%cd sd-scripts 
!pip install --upgrade -r requirements.txt 

!apt-get -y install -qq aria2
!pip install --pre -U xformers
!pip install -q --pre triton

# 底模型下载，根据需要下载自己的底模型
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/11745 -d /content/models -o model.safetensors

In [ ]:
#@title # 挂载 Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown # 训练参数设置
#@markdown **底模型路径**
pretrained_model = "/content/models/model.safetensors" #@param {type:"string"}
#@markdown **训练数据集路径**
train_data_dir = "/content/drive/MyDrive/ai/" #@param {type:"string"}

#@markdown ---
#@markdown ## 输出设置
#@markdown **模型保存名称**
output_name="" #@param {type:"string"}
#@markdown **模型输出路径** 
output_dir = "/content/drive/MyDrive/ai/outputs/" #@param {type:"string"}
#@markdown **模型保存格式** 
save_model_as="safetensors" #@param ["ckpt", "pt", "safetensors"]

#@markdown ---
#@markdown ## 训练相关参数
#@markdown **图片分辨率，支持非正方形，但必须是 64 倍数**
resolution="512,512" #@param {type:"string"}
#@markdown **batch size**
batch_size=1 #@param {type:"number"}
#@markdown **最大训练 epoch**
max_train_epoches=10 #@param {type:"number"}
#@markdown **每N个 epoch 保存一次**
save_every_n_epochs=2 #@param {type:"number"}
#@markdown **network dim 常用 4~128，不是越大越好**
network_dim=128 #@param {type:"number"}
#@markdown **network alpha**
# 用于控制network dim的权重，通常是network_dim的一半 防止下溢。默认值为 1，使用较小的 alpha 需要提升学习率。
network_alpha=64 #@param {type:"number"}
clip_skip=2 # 一般用 2
#@markdown **仅训练 unet，开启这个会牺牲效果大幅减少显存使用。6G显存可以开启**
train_unet_only=0 #@param {type:"number"}
#@markdown **仅训练 文本编码器**
train_text_encoder_only=0 #@param {type:"number"}
#@markdown **seed，跑测试使用**
seed="1123" #@param {type:"string"}
#@markdown **noise_offset (0-1)**
noise_offset="0.1" #@param {type:"string"}

#@markdown ---
#@markdown ## Learning rate 
#@markdown **学习率**
lr="6e-5" #@param {type:"string"}
#@markdown **unet 学习率**
unet_lr="6e-5" #@param {type:"string"}
#@markdown **text encoder 学习率**
text_encoder_lr="7e-6" #@param {type:"string"}
# 
lr_scheduler="cosine_with_restarts" #@param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"]
#@markdown **仅在 lr_scheduler 为 constant_with_warmup 时需要填写这个值**
lr_warmup_steps=0 #@param {type:"number"}
#@markdown **余弦退火重启次数,仅在 lr_scheduler 为 cosine_with_restarts 时起效。** 
lr_restart_cycles=1 #@param {type:"number"}

#@markdown ---
#@markdown ## 其他设置
#@markdown **若需要从已有的 LoRA 模型上继续训练，请填写 LoRA 模型路径。**
network_weights="" #@param {type:"string"}
#@markdown **arb 最小分辨率**
min_bucket_reso=256 #@param {type:"number"}
#@markdown **arb 最大分辨率**
max_bucket_reso=1024 #@param {type:"number"}
persistent_data_loader_workers=0 # 容易爆内存，保留加载训练集的worker，减少每个 epoch 之间的停顿

# 优化器设置
# lr="1e-4" unet_lr="1e-4" text_encoder_lr="1e-5"
use_8bit_adam=0 # 使用 8bit adam 优化器节省显存，默认启用。部分 10 系老显卡无法使用，修改为 0 禁用。
use_lion=1 # 使用 Lion 优化器

In [ ]:
%cd /content/sd-scripts/
!accelerate launch --num_cpu_threads_per_process=8 "train_network.py" \
  --enable_bucket \
  --pretrained_model=$pretrained_model \
  --train_data_dir=$train_data_dir \
  --output_dir=$output_dir \
  --logging_dir="./logs" \
  --resolution=$resolution \
  --network_module=networks.lora \
  --max_train_epochs=$max_train_epoches \
  --learning_rate=$lr \no
  --unet_lr=$unet_lr \
  --text_encoder_lr=$text_encoder_lr \
  --lr_scheduler=$lr_scheduler \
  --lr_warmup_steps=$lr_warmup_steps \
  --lr_scheduler_num_cycles=$lr_restart_cycles \
  --network_dim=$network_dim \
  --network_alpha=$network_alpha \
  --output_name=$output_name \
  --train_batch_size=$batch_size \
  --save_every_n_epochs=$save_every_n_epochs \
  --mixed_precision="fp16" \
  --save_precision="fp16" \
  --seed=$seed \
  --noise_offset=$noise_offset \
  --cache_latents \
  --clip_skip=$clip_skip \
  --prior_loss_weight=1 \
  --max_token_length=225 \
  --caption_extension=".txt" \
  --save_model_as=$save_model_as \
  --min_bucket_reso=$min_bucket_reso \
  --max_bucket_reso=$max_bucket_reso \
  --xformers --shuffle_caption 
  --use_8bit_adam
  # --use_lion_optimizer